<a href="https://colab.research.google.com/github/hivancd/ehealth-entity-recognition-problem/blob/franco/myTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Innit

In [2]:
#Innit
! pip install gliner
! pip install accelerate -U
! git clone https://github.com/hivancd/ehealth-entity-recognition-problem
! git clone https://github.com/ehealthkd/corpora
import sys
sys.path.append('ehealth-entity-recognition-problem')
from anntools import Collection
from pathlib import Path
! pip install spacy
! python -m spacy download es_core_news_sm
import spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 714.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinu

### Loading trainning data

In [ ]:
c = Collection()
c.load(Path('/content/ehealth-entity-recognition-problem/data/training/scenario.txt'))

In [ ]:
# def get_ner(tokens:list,keyphrases):



In [ ]:
# HIVAN
tokenizer = spacy.load("es_core_news_sm")
data=[]
in_dict={'tokenized_sentence':[],'ner':[]}
# for sentence in c.sentences:
#   text= sentence.text
#   tokens = tokenizer(text)
#   in_dict['tokenized_sentence']= tokens

s=c.sentences[0]
t=tokenizer()
kp=s.keyphrases[0]
print(kp.spans)
# for t in tokens:

[(51, 61)]


### Testing/Evaluating data

In [3]:
# FRANCO
! git clone https://github.com/ehealthkd/corpora
# Import libraries
from gliner import GLiNER
import csv
# Load the pre-trained GLiNER model
model = GLiNER.from_pretrained("urchade/gliner_mediumv2.1")
# model.eval()  # Set the model to evaluation mode

# Load your data from the specified path
# c = Collection()
# c.load(Path('/content/ehealth-entity-recognition-problem/data/testing/scenario2-taskA/scenario.txt'))

# Access the sentences in the Collection
# sentences = c.sentences  # This should give you the list of Sentence objects
# print(sentences)

# Labels for entity prediction
labels = ["Concept","Action","Predicate"]

text ="""Todas las personas de seis meses de edad o mayores deben vacunarse cada temporada contra la gripe.
Las talasemias son más frecuentes entre las personas de ascendencia italiana, griega, del Medio Oriente, del sur de Asia y africana.
Beber mucho líquido, descansar y tomar analgésicos sin aspirina puede ayudar.
La insulina es una hormona que ayuda a la glucosa a entrar a las células para darles energía.
El útero o matriz es el lugar donde se desarrolla el bebé cuando una mujer está embarazada.
El síndrome de dolor regional complejo es un trastorno de dolor crónico.
Un implante coclear es un dispositivo electrónico pequeño y complejo que puede ayudar a proporcionar algún sentido de la audición.
Las piernas están formadas por huesos, vasos sanguíneos, músculos y otros tejidos conectivos.
Aproximadamente 1 de cada 5.000 personas tiene EDS.
Algunas personas tienen solo un ataque de vértigo de vez en cuando y otras pueden tener ataques más frecuentemente  durante varios días.
Si los gérmenes permanecen en la nariz y en la garganta del niño, probablemente no se enferme.
El tejido que recubre los pulmones, el estómago, el corazón y otros órganos se llama mesotelio.
El trastorno de pánico es un tipo de trastorno de ansiedad.
El virus es contagioso y puede diseminarse de persona a persona.
También puede convertirse en síndrome de shock por dengue que causa sangrado masivo y shock.
Los tratamientos incluyen cremas, medicinas y fototerapia.
Dejar de fumar y evitar el consumo del alcohol puede ayudar.
También puede afectar los tobillos, talones, rodillas, muñecas, dedos y codos.
Si está embarazada, la sífilis puede causar defectos congénitos o abortos.
Pero a veces los gérmenes pasan a los pulmones o al torrente sanguíneo y entonces el Hib puede causar problemas graves como la meningitis y la neumonía.
LASIK - queratomileusis in situ asistida por rayos láser - es una de las más comunes.
Hay dos tipos, grupo A y grupo B.
Ocurre cuando el cuerpo tiene una abrumadora respuesta inmunitaria a una infección bacteriana.
A muchas personas les avergüenza hablar acerca de problemas  rectales.
Las hemorroides son venas hinchadas, inflamadas alrededor del ano o la parte inferior del recto.
El virus sólo infecta a los seres humanos, no es el mismo parvovirus que pueden tener los perros o los gatos.
Esto reduce el riesgo de contraer enfermedades del corazón.
El síndrome de muerte súbita infantil  es la muerte repentina e inexplicable de un niño menor de un año de edad.
El consumo de tabaco o de alcohol aumenta el riesgo.
La cafeína, el tabaco y el alcohol pueden empeorar los síntomas.
Sólo el 25 por ciento de los niños tienen síntomas de ERGE.
La inserción de un aparato en el oído externo que impulsa aire al oído medio puede ayudar.
La esclerosis tuberosa no tiene cura, pero los tratamientos pueden ayudar a disminuir los síntomas.
El médico puede realizar pruebas que detecten si padece de arritmia.
Varían desde decir las palabras de manera incorrecta hasta la incapacidad total para hablar o entender el habla.
Puede incluir cirugía, radioterapia, quimioterapia o terapia biológica.
Un menor de edad con el virus puede presentar fiebre, obstrucción nasal, tos y dificultades para respirar.
Uno de cada 10 recién nacidos en los Estados Unidos es prematuro.
No se conocen las causas del trastorno del espectro autista.
Los resfríos raramente causan fiebre o dolores de cabeza.
Después de una lesión, una enfermedad o una cirugía seria, la recuperación puede ser lenta.
Un tipo de endoscopía que analiza el intestino grueso se llama colonoscopía.
Las caderas son muy estables.
También muestran qué tan bien funcionan las cámaras y válvulas de su corazón.
Es posible que necesite una dieta especial para reducir la cantidad de potasio que consume.
La malaria es una enfermedad grave causada por un parásito.
Una vez que el CMV penetra en el cuerpo de la persona, permanece ahí para siempre.
Los tumores están formados por células extras.
Otras causas incluyen los nódulos tiroideos, la tiroiditis, el consumo excesivo de yodo y tomar demasiada hormona tiroidea sintética.
También es importante reducir el riesgo de propagación del VIH a otras personas.
Esto es debido a que diferentes tejidos absorben diferentes cantidades de radiación.
La enfermedad de Creutzfeldt-Jakob (CJD, por sus siglas en inglés) es un trastorno cerebral degenerativo poco común.
Esta inflamación puede ser en uno o en los dos lados de la cara.
Sin embargo, los genes y las experiencias de la niñez pueden representar un papel importante.
Es probable que le hagan un examen de sangre, incluyendo pruebas de función hepática y análisis de sangre.
La disfunción eréctil (DE) es una disfunción sexual masculina común.
No obstante, las células de los islotes trasplantadas pueden continuar el trabajo de las células destruidas.
Puede ser una lumpectomía o una mastectomía.
A menudo, una cirugía puede cerrar el labio y el paladar.
Una muestra de materia fecal puede diagnosticarla.
Las estatinas son drogas usadas para bajar el colesterol.
Sin la insulina, hay un exceso de glucosa que permanece en la sangre.
En las personas con sistemas inmunitarios debilitados, la EAG puede causarles problemas más serios.
La urticaria es muy común.
Un análisis de sangre puede determinar si tiene el virus.
Suele tomar varios años para que las células normales del cuello uterino se conviertan en células cancerosas.
El hipotiroidismo es más común en las mujeres, en las personas con otros problemas de la tiroides y en las personas mayores de 60 años de edad.
Las personas con alto riesgo de deficiencia de vitamina D pueden necesitar más.
La pancreatitis es una inflamación del páncreas.
Indica que el corazón no puede bombear la sangre de la manera que debería hacerlo.
Los animales de granja pueden transmitir enfermedades.
Un Consejo Institucional de Revisión (Institutional Review Board, IRB) revisa, supervisa y aprueba muchos de los ensayos clínicos.
Este daño se denomina neuropatía diabética.
Los planes de administración de seguros de salud son un tipo de seguro de salud.
Las mamografías pueden encontrar algo que no sea cáncer.
Otro signo es un ataque isquémico transitorio, un "mini accidente cerebrovascular".
Las personas con este síndrome sufren de anormalidades faciales que incluyen ojos muy separados y estrechos, problemas de crecimiento y anomalías del sistema nervioso.
Suministran sangre al cerebro y a la cabeza.
Las hormonas son mensajeros químicos.
Los hábitos saludables, incluyendo una alimentación nutritiva y la actividad física, pueden ayudarte a sentirte bien, verte bien y a esforzarte en la escuela, el trabajo o los deportes.
En muy pocos casos, procedimientos médicos y cirugía también pueden ser usados en niños cuyas arterias coronarias están comprometidas.
Esto sucede cuando los glóbulos rojos son destruidos antes que el cuerpo pueda reponerlos.
La grasa es un tipo de nutriente.
El cáncer más común en los niños es la leucemia.
Los lunares son muy comunes.
La oxigenoterapia hiperbárica es otro tipo de terapia con oxígeno.
El intestino delgado forma parte del sistema digestivo.
Los piojos no transmiten enfermedades.
La tricomoniasis también puede causar vaginitis.
La deficiencia de glucosa -6- fosfato-deshidrogenasa (también llamada deficiencia de G-6-PD) es una enfermedad hereditaria que se da generalmente en los varones.
La cirugía corta, anuda o sella las trompas de Falopio.
Puede sentirse aturdido o no sentirse bien por varios días o semanas después de la lesión.
En la diabetes tipo 1, el páncreas no produce insulina.
Los niveles de colesterol tienden a aumentar con la edad.
Los animales salvajes suelen huir de las personas.
Las inmunizaciones son una manera más fácil y menos riesgosa de hacerse inmune.
Si los resultados son anormales, el doctor puede recomendar otras pruebas como una biopsia.
Una caída brusca de la presión arterial o estar deshidratado puede hacer que se sienta mareado.
Los análisis de sangre pueden ayudar al médico a diagnosticar la enfermedad.
La tendinitis del codo es una lesión deportiva, con frecuencia por jugar tenis o golf."""


# Perform entity prediction
entities_list = model.predict_entities(text, labels, threshold=0.0001)

# Display predicted entities and their labels
for entity in entities_list:
    print(entity["text"], "=>", entity["label"])

# Save predictions in the required format
with open('predictions.ann', 'w') as f:
    id_counter = 1  # Initialize a counter for entity IDs
    for entity in entities_list:
        # Ensure the entity has the necessary fields
        if 'start' in entity and 'end' in entity and 'label' in entity and 'text' in entity:
            # Write the predictions in the required format with IDs
            f.write(f"T{id_counter}\t{entity['label']} {entity['start']} {entity['end']} {entity['text']}\n")
            id_counter += 1  # Increment the counter for the next entity
        else:
            print(f"Skipping malformed entity: {entity}")  # Log the malformed entity

# Function to convert .ann predictions to .txt format
def convert_ann_to_txt(ann_file_path, txt_file_path):
    with open(ann_file_path, 'r') as ann_file, open(txt_file_path, 'w') as txt_file:
        for line in ann_file:
            # Assuming the line format is: T{id} <label> <start> <end> <text>
            parts = line.strip().split('\t')  # Split by tab for T{id} format
            if len(parts) == 2:  # Ensure there are two parts: ID and the rest
                id_label, rest = parts
                rest_parts = rest.split()  # Split the rest into components
                if len(rest_parts) >= 3:  # Ensure there are at least 3 components
                    label = rest_parts[0]
                    start = rest_parts[1]
                    end = rest_parts[2]
                    text = ' '.join(rest_parts[3:])  # Join the rest as text
                    # Write to the txt file in the expected format with IDs
                    txt_file.write(f"{id_label}\t{label} {start} {end} {text}\n")



# Example usage
convert_ann_to_txt('predictions.ann', 'predictions.txt')
convert_ann_to_txt('/content/ehealth-entity-recognition-problem/data/testing/scenario1-main/scenario.ann', 'scenario.txt')
# convert_ann_to_csv('/content/ehealth-entity-recognition-problem/data/testing/scenario2-taskA/scenario.ann', 'scenario.csv')



fatal: destination path 'corpora' already exists and is not an empty directory.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:206: UserWarning: Sentence of length 1467 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Todas las personas de seis meses de edad o mayores => Predicate
vacunarse => Action
temporada => Action
gripe => Concept
talasemias => Concept
personas de ascendencia italiana => Predicate
griega => Predicate
Medio Oriente => Concept
sur de Asia => Concept
africana => Predicate
Beber mucho líquido => Action
descansar => Action
y => Action
tomar analgésicos => Action
sin => Action
aspirina => Predicate
puede ayudar => Action
insulina => Predicate
hormona => Concept
ayuda => Action
glucosa => Concept
entrar => Action
células => Concept
energía => Concept
El útero o matriz => Concept
bebé => Concept
mujer => Predicate
embarazada => Concept
El síndrome de dolor regional complejo => Concept
trastorno de dolor crónico => Concept
implante coclear => Predicate
dispositivo electrónico => Concept
pequeño => Concept
complejo => Concept
ayudar => Action
proporcionar => Action
sentido => Concept
audición => Concept
Las piernas => Concept
huesos => Concept
vasos sanguíneos => Concept
músculos => Con

## Mauricio

In [ ]:
a = [[print("'"+x["text"]+"'"), [print(b.text,"->", b.label) for b in x.keyphrases],print()] for x in c.sentences[:10]]

TypeError: 'Sentence' object is not subscriptable